# Welcome to DeTTECT

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/th3y3ti/DeTTECT/master)

In [2]:
!python dettect.py -h

usage: dettect.py [-h] [--version] [-i]  ...

Detect Tactics, Techniques & Combat Threats

optional arguments:
  -h, --help         show this help message and exit
  --version          show program's version number and exit
  -i, --interactive  launch the interactive menu, which has support for all
                     modes

MODE:
  Select the mode to use. Every mode has its own arguments and help info
  displayed using: {editor, datasource, visibility, detection, group,
  generic} --help

  
    editor (e)       DeTT&CT Editor
    datasource (ds)  data source mapping and quality
    visibility (v)   visibility coverage mapping based on techniques and data
                     sources
    detection (d)    detection coverage mapping based on techniques
    group (g)        threat actor group mapping
    generic (ge)     includes: statistics on ATT&CK data source and updates on
                     techniques, groups and software

Source: https://github.com/rabobank-cdc/DeTTECT


# Data Sources

<hr>


##  Metrics
We can use DeTTECT to collect some basic metrics on the number of unique techniques by data source.

In [7]:
!python dettect.py generic --datasources

Count  Data Source
--------------------------------------------------
178    Process monitoring
107    File monitoring
103    Process command-line parameters
49     API monitoring
39     Process use of network
37     Packet capture
36     Windows Registry
30     Authentication logs
28     Netflow/Enclave netflow
26     Windows event logs
22     Network protocol analysis
18     DLL monitoring
18     Binary file metadata
17     Azure activity logs
15     AWS CloudTrail logs
13     Stackdriver logs
13     Loaded DLLs
10     SSL/TLS inspection
9      PowerShell logs
9      Network intrusion detection system
9      System calls
9      Malware reverse engineering
8      Anti-virus
8      Network device logs
7      Office 365 account logs
7      Application logs
7      Kernel drivers
6      Data loss prevention
5      Web proxy
5      Email gateway
4      Mail server
4      Web logs
4      Services
4      Windows Error Reporting
4      User interface
4      Host network interface
3      DNS r

## Administration File

We need to start by creating a data source administration file. Administration files are written in the YAML text format. Example administration files can be found in `./sample-data`.

In [4]:
!python dettect.py datasource -h

usage: dettect.py datasource [-h] [-ft FILE_TECH] -fd FILE_DS [-s SEARCH] [-l]
                             [-e] [-g] [-y] [-ya] [-u] [--health]

Create a heat map based on data sources, output data sources to Excel or
generate a data source improvement graph.

optional arguments:
  -h, --help            show this help message and exit
  -ft FILE_TECH, --file-tech FILE_TECH
                        path to the technique administration YAML file (used
                        with the option '-u, --update' to update the
                        visibility scores)
  -fd FILE_DS, --file-ds FILE_DS
                        path to the data source administration YAML file
  -s SEARCH, --search SEARCH
                        only include data sources which match the provided EQL
                        query
  -l, --layer           generate a data source layer for the ATT&CK navigator
  -e, --excel           generate an Excel sheet with all data source
  -g, --graph           generate a graph wi



### Validate YAML Syntax

I created a data source administration file for Carbon Black Response. Normally CbR would just be a `product` in a data sources administration file, but this is helpful to gauge the data sources and technique visibility that it can provide. 

We can confirm that the file conforms to the proper syntax by running the `--health` command. It will only generate output if there is an error.

In [16]:
!python dettect.py datasource -fd ./sample-data/data-sources-cbr-response.yaml --health

[!] EMPTY or INVALID value for 'platform' within the data source admin. file: Foo (should be value(s) of: [Windows, Linux, macOS, AWS, GCP, Azure, Azure AD, Office 365, SaaS] or all)


Woops, I included an invalid `platform` in the data source administration file. Once we fix that, it should not print anything to `stdout`.

### Generate ATT&CK Navigator Layers

Once the data sources administration file has passed the health check, we can create an ATT&CK navigator layer from it:

In [4]:
!python dettect.py datasource -fd ./sample-data/data-sources-cbr-response.yaml -l

File written:   output/data_sources_carbon-black-response.json


In [9]:
from pathlib import Path
import json

navigator_layer = json.load(Path('./output/data_sources_carbon-black-response.json').open())
navigator_layer

{'name': 'Data sources carbon-black-response',
 'version': '2.2',
 'domain': 'mitre-enterprise',
 'description': 'description',
 'filters': {'stages': ['act'], 'platforms': ['Windows', 'Linux', 'macOS']},
 'sorting': 0,
 'viewMode': 0,
 'hideDisable': False,
 'techniques': [{'techniqueID': 'T1531',
   'color': '#AB47BC',
   'comment': '',
   'enabled': True,
   'tactic': 'impact',
   'metadata': [{'name': '-Available data sources',
     'value': 'Process monitoring, Process command-line parameters'},
    {'name': '-ATT&CK data sources',
     'value': 'Process command-line parameters, Windows event logs, Process monitoring'},
    {'name': '-Products', 'value': 'cbapi.response.models.Process'}]},
  {'techniqueID': 'T1529',
   'color': '#AB47BC',
   'comment': '',
   'enabled': True,
   'tactic': 'impact',
   'metadata': [{'name': '-Available data sources',
     'value': 'Process monitoring, Process command-line parameters'},
    {'name': '-ATT&CK data sources',
     'value': 'Process com

We can take our navigator layer and load MITRE's ATT&CK Navigator.

### Generate Excel Spreadsheet

We can also create our data source administration file as an Excel spreadsheet:

In [10]:
!python dettect.py datasource -fd ./sample-data/data-sources-cbr-response.yaml -e

File written:   output/data_sources.xlsx


### Generate Technique Administration File

Once we update our data source administration file, we can use that to generate the administration file for techniques (visibility/detection).

In [11]:
!python dettect.py datasource -fd ./sample-data/data-sources-cbr-response.yaml -y

File written:   output/techniques-administration-carbon-black-response-windows-linux-macos.yaml
